# Contents
1. Tensor related methods<br>
    1.1. Tensor & Variable<br>
    1.2. Tensor의 자료형<br>
    1.3. torch.tensor<br>
    1.4. Tensor 관련 함수와 메서드들<br>
    - Refer to [여기1](https://pytorch.org/blog/pytorch-0_4_0-migration-guide/), [여기2](https://subinium.github.io/pytorch-Tensor-Variable/https://pytorch.org/blog/pytorch-0_4_0-migration-guide/), [여기3](https://subinium.github.io/pytorch-Tensor-Variable/)
2. Tensor Random<br>
    2.1. Pytorch Randomness<br>
    2.2. CuDNN<br>
    2.3. Numpy<br>
    2.4. Random<br>
    2.5. Usage of torch random<br>
    - Refer to [여기](https://hoya012.github.io/blog/reproducible_pytorch/https://hoya012.github.io/blog/reproducible_pytorch/)
3. train(), eval(), no_grad()

## Tensor related methods

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.__version__

'1.1.0'

In [3]:
import os
import sys

In [4]:
sys.version_info

sys.version_info(major=3, minor=6, micro=12, releaselevel='final', serial=0)

### Tensor와 Variable

<u>Tensor</u>와 <u>variable</u>은 `Tensor`로 통합. Tensor는 기본적으로 NumPy 배열과 동일.

### Tensor의 자료형
CPU이냐 GPU이냐에 따라 구분

- CPU tensor: torch.FloatTensor
- GPU tensor: torch.cuda.FloatTensor

In [5]:
cpu_tensor = torch.Tensor([1,2])
gpu_tensor = torch.Tensor([1,2]).cuda()

In [6]:
cpu_tensor.device

device(type='cpu')

In [7]:
gpu_tensor.device

device(type='cuda', index=0)

### About torch.Tensor

In [8]:
x = torch.tensor([1,2])
x

tensor([1, 2])

* `.data`: array형태의 데이터. 원래는 Variable의 속성이었으나, 통합된 이후 잘 사용되지 않음. 대신 .detatch()사용
    * .data는 잘못된 gradient에도 update에 사용(.data는 사용하지 않는 것이 좋음)
* `dtype`: 데이터 유형
* `device`: 장치 유형(cpu vs cuda). data를 GPU 혹은 CPU 버퍼에 옮김
    * 중간에 데이터를 보거나 아니면 test할 때 .cpu(), .detatch() 등을 사용하는데 이는 텐서에 대한 일부작업은 cuda 텐서에서 수행할 수 없으므로 먼저 cpu로 이동하여 계산되어야 함(cuda를 쓰는 것은 어쨋든 학습에서)
* `requires_grad`: gradient 값 저장 유무(default: False)
* `pin_memory`: True시에 pinned memory 할당, CPU tensor에서 가능.
    * <u><b>CPU에서 메모리를 할당하면 GPU 메모리로 바로 복사가 불가능</b></u>하다. 따라서 CPU에서 할당한 메모리를 GPU로 복사하기 위해서 GPU driver가 Pinned Memory를 할당
    * 참고: https://mkblog.co.kr/2017/03/07/nvidia-gpu-pinned-host-memory-cuda/

In [9]:
x.data

tensor([1, 2])

In [10]:
x.dtype

torch.int64

In [11]:
x.device

device(type='cpu')

In [12]:
x.requires_grad

False

In [13]:
x.pin_memory

<function Tensor.pin_memory>

### Tesnor관련 함수와 메서드들

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
device

'cuda'

In [16]:
x = torch.tensor([2, 4], dtype=torch.int32, device = device)

In [17]:
x

tensor([2, 4], device='cuda:0', dtype=torch.int32)

.to()를 이용하여 바꿀 수 있음

In [18]:
x = x.to('cpu', dtype=torch.uint8)

In [19]:
x

tensor([2, 4], dtype=torch.uint8)

detatch() & clone()<br>
* `detatch()`: 기존 Tensor에서 gradient 전파가 안되는 Tensor 생성
    * Tensord의 requires_grad = False, grad_fn = None
    * with torch.no_grad()와 동일
* `clone()`: 기존 Tensor와 내용을 복사한 텐서 생성

view() vs reshape()
- 참고: https://sanghyu.tistory.com/3

* `view()`: 새로운 모양의 tensor를 반환. 반환된 tensor는 원본 tensor와 기반이 되는 data를 공유. 변환된 tensor 값 변경 -> viewed되는 tensor 값 또한 변경
* `reshape()`: 원본 tensor의 복사본 혹은 view를 반환. copy를 받을지 view를 받을지 모름. 따라서 clone()을 이용하여 copy하거나 view()를 이용 -> 결론은 view()를 쓰면 됨

transpose() vs permute()

* `transpose()`: 2개의 차원을 변경
* `permute()`: 모든 차원의 순서를 재배치

In [20]:
x = torch.rand(1, 2, 3)
x

tensor([[[0.8672, 0.2057, 0.3760],
         [0.1571, 0.3096, 0.7961]]])

In [21]:
y = x.transpose(0, 2) # 0<-> 2 차원 변경, [3, 2, 1]
y

tensor([[[0.8672],
         [0.1571]],

        [[0.2057],
         [0.3096]],

        [[0.3760],
         [0.7961]]])

In [22]:
z = x.permute(2, 1, 0)  # 0->2, 1->1, 2->0으로 차원 변경, [3, 2, 1]
z

tensor([[[0.8672],
         [0.1571]],

        [[0.2057],
         [0.3096]],

        [[0.3760],
         [0.7961]]])

## Tensor Random

In [23]:
import numpy as np
import random

기본 random settings for reproducible

In [24]:
random_seed = 777

In [25]:
torch.manual_seed(random_seed)
if device == 'cuda':
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

### pytorch randomness

`torch.`에서 발생하는 randomness들이 통제<br>
단, nondeterministic 존재.<br>
* <u>nondeterministic list</u>
    * forward kernel
        * torch.Tensor.index_add_()
        * torch.Tensor.scatter_add_()
        * torch.bincount()
    * backward kernel
        * torch.nn.functional.embedding_bag()
        * torch.nn.functional.ctc_loss()
        * torch.nn.functional.interpolate()

In [26]:
torch.manual_seed(random_seed)

### CuDNN

CuDNN 관련 Randomness 제어설정. 단, 연산 처리 속도가 감소되는 문제 발생. 

In [27]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Numpy

numpy로 처리되는 데이터 전처리, load 등등에 사용됨

In [28]:
np.random.seed(random_seed)

### Random

torchvision에서 transforms 등과 같은 randomvess는 torch, numpy, cudnn이 아닌 python random 라이브러리에 의해 randomness가 결정

In [29]:
random.seed(random_seed)

### Usage of torch random

In [30]:
torch.rand(5) # 0~1 사이의 숫자 균등하게(uniform) 생성

tensor([0.0819, 0.4911, 0.4033, 0.3859, 0.8813])

In [31]:
torch.randn(5) # 평균이 0, 표준편차가 1인 가우시안 정규분포를 이용해 생성

tensor([0.2197, 0.0695, 1.3153, 1.0535, 2.2013])

In [32]:
torch.randint(0, 10, (1,2)) # 주어진 범위 내 정수를 균등하게 생성

tensor([[3, 0]])

In [33]:
torch.randperm(11) # 0~n-1 범위 내 임의의 순열을 반환

tensor([ 8, 10,  6,  5,  1,  0,  4,  3,  7,  2,  9])

_like(): 같은 사이즈를 생성

In [34]:
x = torch.rand(5)
x

tensor([0.3154, 0.3771, 0.6166, 0.5355, 0.9970])

In [35]:
torch.rand_like(x)

tensor([0.3777, 0.9541, 0.8282, 0.6459, 0.5544])

## train(), eval(), no_grad()

* `.train()`: 모든 레이어에 train mode 선언. 파라미터들이 학습됨
* `.eval()`: 모든 레이어에 eval mode 선언. 배치 정규화 dropout layer들은 학습 모드 대신 eval mode로 사용(eval에서는 dropout, 배치 정규화는 비활성화)
* `no_grad()`: 역전파 비활성화